<a href="https://colab.research.google.com/github/aogara-ds/reason/blob/main/Classification_project_skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before running, choose Runtime > Change runtime type and select GPU.


# Setup



In [1]:
!pip install transformers
!pip install torch

     |████████████████████████████████| 4.2 MB 3.2 MB/s 
     |████████████████████████████████| 84 kB 854 kB/s 
     |████████████████████████████████| 6.6 MB 35.5 MB/s 
     |████████████████████████████████| 596 kB 42.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import json

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)
model.eval().cuda()

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

# GPT-2 Utility functions

Resources:

1.   https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel
2.   https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-generation/run_generation.py




In [ ]:
def generate(prompt, max_length=5, stop_token=None):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    generated_text_ids = model.generate(input_ids=input_ids.cuda(), max_length=max_length+len(input_ids[0]), do_sample=False)
    generated_text = tokenizer.decode(generated_text_ids[0], clean_up_tokenization_spaces=True)
    post_prompt_text = generated_text[len(tokenizer.decode(input_ids[0], clean_up_tokenization_spaces=True)):]
    return prompt + post_prompt_text[:post_prompt_text.find(stop_token) if stop_token else None]

In [ ]:
# Note that the logits are shifted over 1 to the left, since HuggingFace doesn't give a logit for the first token
def get_logits_and_tokens(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    tokens = [tokenizer.decode([input_id]) for input_id in input_ids[0]]
    output = model(input_ids.cuda())
    return output.logits[0][:-1], tokens

# Example calls to GPT-2 Utility functions

In [3]:
EXAMPLE_PROMPT = """Horrible: negative
Great: positive
Bad:"""

generated_text = generate(EXAMPLE_PROMPT, stop_token="\n")
generated_text

NameError: ignored

In [ ]:
logits, tokens = get_logits_and_tokens(generated_text)
last_token_probs = torch.softmax(logits[-1], dim=0)
negative_prob = last_token_probs[tokenizer.encode(" negative")[0]]
positive_prob = last_token_probs[tokenizer.encode(" positive")[0]]

print(f"tokens: {tokens}\nnegative prob: {negative_prob}\npositive prob: {positive_prob}")

# Loading the data

Note: before running this section load `train.jsonl` into the runtime

In [4]:
def load_jsonl(filename):
    f = open(filename)
    return [json.loads(line) for line in f.read().splitlines()]

In [5]:
train_examples = load_jsonl("train.jsonl")
train_examples[-1]

FileNotFoundError: ignored

# Basic prompt building

In [ ]:
def render_example(example):
    title = example["text"].split(".")[0].strip()
    abstract = example["text"][len(title)+1:].strip()
    return f"""Title: {title}
Abstract: {abstract}
Label: {"AI" if example["label"] == "True" else "Not AI"}"""

In [ ]:
def render_end_example(example):
    title = example["text"].split(".")[0].strip()
    abstract = example["text"][len(title)+1:].strip()
    return f"""Title: {title}
Abstract: {abstract}
Label:"""

In [ ]:
def make_prompt(instructions, train_examples, end_example):
    rendered_train_examples = "\n\n--\n\n".join([render_example(example) for example in train_examples])
    return f"""{instructions}

{rendered_train_examples}

--

{render_end_example(end_example)}"""

In [ ]:
INSTRUCTIONS = "Classify the following examples based on whether they are AI-relevant or not:"

prompt = make_prompt(INSTRUCTIONS, train_examples[:4], train_examples[4])
print(prompt)

Classify the following examples based on whether they are AI-relevant or not:

Title: thermodynamic analysis of quantum error correcting engines
Abstract: quantum error correcting codes can be cast in a way which is strikingly similar to a quantum heat engine undergoing an otto cycle. in this paper we strengthen this connection further by carrying out a complete assessment of the thermodynamic properties of strokes operator based error correcting codes. this includes an expression for the entropy production in the cycle which as we show contains clear contributions stemming from the different sources of irreversibility. to illustrate our results we study a classical qubit error correcting code well suited for incoherent states and the qubit shor code capable of handling fully quantum states. we show that the work cost associated with the correction gate is directly associated with the heat introduced by the error. moreover the work cost associated with encoding decoding quantum informa

In [ ]:
generated_text = generate(prompt, stop_token="\n")
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Classify the following examples based on whether they are AI-relevant or not:

Title: thermodynamic analysis of quantum error correcting engines
Abstract: quantum error correcting codes can be cast in a way which is strikingly similar to a quantum heat engine undergoing an otto cycle. in this paper we strengthen this connection further by carrying out a complete assessment of the thermodynamic properties of strokes operator based error correcting codes. this includes an expression for the entropy production in the cycle which as we show contains clear contributions stemming from the different sources of irreversibility. to illustrate our results we study a classical qubit error correcting code well suited for incoherent states and the qubit shor code capable of handling fully quantum states. we show that the work cost associated with the correction gate is directly associated with the heat introduced by the error. moreover the work cost associated with encoding decoding quantum informa